# Swap HDX
Agent $\alpha$ swaps asset $i$ with the protocol for HDX.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))

import inspect
from model.amm.omnipool_amm import swap_hdx, swap_hdx_delta_Qi, swap_hdx_delta_Ri

## Constraints

The swap will keep the swap invariant for asset $i$, $R_iQ_i$, constant.

## Requirements

#### Case 1: Asset $i$ sold
$$
\Delta R_i \leq r_i^\alpha
$$
#### Case 2: HDX sold
$$
\Delta Q_i \leq q^\alpha
$$

## Updating AMM state

### Case 1: Asset $i$ sold

$$
\Delta Q_i = Q_i\frac{- \Delta R_i}{R_i + \Delta R_i}
$$

In [2]:
print(inspect.getsource(swap_hdx_delta_Qi))

def swap_hdx_delta_Qi(old_state: dict, delta_Ri: float, i: int) -> float:
    return old_state['Q'][i] * (- delta_Ri / (old_state['R'][i] + delta_Ri))



### Case 2: HDX sold

$$
\Delta R_i = R_i\frac{- \Delta Q_i}{Q_i + \Delta Q_i}
$$

In [3]:
print(inspect.getsource(swap_hdx_delta_Ri))

def swap_hdx_delta_Ri(old_state: dict, delta_Qi: float, i: int) -> float:
    return old_state['R'][i] * (- delta_Qi / (old_state['Q'][i] + delta_Qi))



## Updating agent state

$$
\Delta q^\alpha = - \Delta Q_i\\
\Delta r_i^\alpha = - \Delta R_i
$$

In [4]:
print(inspect.getsource(swap_hdx))

def swap_hdx(
        old_state: dict,
        old_agents: dict,
        trader_id: string,
        delta_R: float,
        delta_Q: float,
        i: int
) -> tuple:
    """Compute new state after HDX swap"""

    new_state = copy.deepcopy(old_state)
    new_agents = copy.deepcopy(old_agents)
    if delta_Q == 0:
        delta_Q = swap_hdx_delta_Qi(old_state, delta_R, i)
    elif delta_R == 0:
        delta_R = swap_hdx_delta_Ri(old_state, delta_Q, i)
    else:
        raise Exception

    # Token amounts update
    new_state['R'][i] += delta_R
    new_state['Q'][i] += delta_Q
    new_agents[trader_id]['r'][i] -= delta_R
    new_agents[trader_id]['q'] -= delta_Q

    return new_state, new_agents

